In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

/home/user/anaconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/user/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 16)

/home/user/anaconda3/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/user/anaconda3/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


In [4]:
# model.set_parameters("trained_models/g1-reptile/sac")

In [5]:
model.learn(
    total_timesteps=int(1e7), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

Logging to ./logs/sac/reptile_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 217       |
|    ep_rew_mean     | -1.45e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 523       |
|    time_elapsed    | 46        |
|    total_timesteps | 24512     |
| train/             |           |
|    actor_loss      | 17.4      |
|    critic_loss     | 2.59      |
|    ent_coef        | 0.863     |
|    ent_coef_loss   | -2.45     |
|    learning_rate   | 0.0001    |
|    n_updates       | 1525      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 206       |
|    ep_rew_mean     | -1.14e+03 |
| time/              |           |
|    episodes        | 200       |
|    fps             | 518       |
|    time_elapsed    | 88        |
|    total_timesteps | 45664     |
| train/             |           |
|    actor_loss      | 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 85.3     |
|    ep_rew_mean     | -273     |
| time/              |          |
|    episodes        | 1600     |
|    fps             | 507      |
|    time_elapsed    | 572      |
|    total_timesteps | 290368   |
| train/             |          |
|    actor_loss      | 169      |
|    critic_loss     | 42.4     |
|    ent_coef        | 0.175    |
|    ent_coef_loss   | -13.7    |
|    learning_rate   | 0.0001   |
|    n_updates       | 18141    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 92.6     |
|    ep_rew_mean     | -293     |
| time/              |          |
|    episodes        | 1700     |
|    fps             | 506      |
|    time_elapsed    | 591      |
|    total_timesteps | 299520   |
| train/             |          |
|    actor_loss      | 171      |
|    critic_loss     | 36.5     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 94       |
|    ep_rew_mean     | -328     |
| time/              |          |
|    episodes        | 3100     |
|    fps             | 501      |
|    time_elapsed    | 863      |
|    total_timesteps | 433328   |
| train/             |          |
|    actor_loss      | 178      |
|    critic_loss     | 62       |
|    ent_coef        | 0.0815   |
|    ent_coef_loss   | -5.04    |
|    learning_rate   | 0.0001   |
|    n_updates       | 27076    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 83.9     |
|    ep_rew_mean     | -301     |
| time/              |          |
|    episodes        | 3200     |
|    fps             | 501      |
|    time_elapsed    | 880      |
|    total_timesteps | 441888   |
| train/             |          |
|    actor_loss      | 178      |
|    critic_loss     | 63.5     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 67.4     |
|    ep_rew_mean     | -224     |
| time/              |          |
|    episodes        | 4600     |
|    fps             | 498      |
|    time_elapsed    | 1093     |
|    total_timesteps | 545424   |
| train/             |          |
|    actor_loss      | 167      |
|    critic_loss     | 56.4     |
|    ent_coef        | 0.0671   |
|    ent_coef_loss   | -0.33    |
|    learning_rate   | 0.0001   |
|    n_updates       | 34082    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 65.9     |
|    ep_rew_mean     | -210     |
| time/              |          |
|    episodes        | 4700     |
|    fps             | 498      |
|    time_elapsed    | 1106     |
|    total_timesteps | 551776   |
| train/             |          |
|    actor_loss      | 165      |
|    critic_loss     | 72.7     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -182     |
| time/              |          |
|    episodes        | 6100     |
|    fps             | 497      |
|    time_elapsed    | 1291     |
|    total_timesteps | 642544   |
| train/             |          |
|    actor_loss      | 154      |
|    critic_loss     | 55       |
|    ent_coef        | 0.0722   |
|    ent_coef_loss   | 0.303    |
|    learning_rate   | 0.0001   |
|    n_updates       | 40152    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 63.1     |
|    ep_rew_mean     | -183     |
| time/              |          |
|    episodes        | 6200     |
|    fps             | 497      |
|    time_elapsed    | 1304     |
|    total_timesteps | 648704   |
| train/             |          |
|    actor_loss      | 154      |
|    critic_loss     | 78.3     |
|    ent_coef 

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.1     |
|    ep_rew_mean     | -29.3    |
| time/              |          |
|    episodes        | 265600   |
|    fps             | 518      |
|    time_elapsed    | 18843    |
|    total_timesteps | 9762896  |
| train/             |          |
|    actor_loss      | 26.6     |
|    critic_loss     | 0.497    |
|    ent_coef        | 0.0591   |
|    ent_coef_loss   | -0.0932  |
|    learning_rate   | 0.0001   |
|    n_updates       | 610174   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.7     |
|    ep_rew_mean     | -28.7    |
| time/              |          |
|    episodes        | 265700   |
|    fps             | 518      |
|    time_elapsed    | 18849    |
|    total_timesteps | 9766080  |
| train/             |          |
|    actor_loss      | 26.8     |
|    critic_loss     | 0.432    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 33.1     |
|    ep_rew_mean     | -28.6    |
| time/              |          |
|    episodes        | 267100   |
|    fps             | 518      |
|    time_elapsed    | 18936    |
|    total_timesteps | 9811536  |
| train/             |          |
|    actor_loss      | 27.3     |
|    critic_loss     | 0.516    |
|    ent_coef        | 0.0591   |
|    ent_coef_loss   | 0.251    |
|    learning_rate   | 0.0001   |
|    n_updates       | 613214   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.2     |
|    ep_rew_mean     | -29.8    |
| time/              |          |
|    episodes        | 267200   |
|    fps             | 518      |
|    time_elapsed    | 18942    |
|    total_timesteps | 9814704  |
| train/             |          |
|    actor_loss      | 26.6     |
|    critic_loss     | 0.411    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.2     |
|    ep_rew_mean     | -28.1    |
| time/              |          |
|    episodes        | 268600   |
|    fps             | 518      |
|    time_elapsed    | 19028    |
|    total_timesteps | 9860192  |
| train/             |          |
|    actor_loss      | 26.6     |
|    critic_loss     | 0.478    |
|    ent_coef        | 0.0591   |
|    ent_coef_loss   | -0.395   |
|    learning_rate   | 0.0001   |
|    n_updates       | 616255   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 33.1     |
|    ep_rew_mean     | -27.8    |
| time/              |          |
|    episodes        | 268700   |
|    fps             | 518      |
|    time_elapsed    | 19035    |
|    total_timesteps | 9863488  |
| train/             |          |
|    actor_loss      | 27.3     |
|    critic_loss     | 0.466    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.2     |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    episodes        | 270100   |
|    fps             | 518      |
|    time_elapsed    | 19121    |
|    total_timesteps | 9908864  |
| train/             |          |
|    actor_loss      | 27.1     |
|    critic_loss     | 0.383    |
|    ent_coef        | 0.0594   |
|    ent_coef_loss   | -0.0659  |
|    learning_rate   | 0.0001   |
|    n_updates       | 619297   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.7     |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    episodes        | 270200   |
|    fps             | 518      |
|    time_elapsed    | 19127    |
|    total_timesteps | 9912112  |
| train/             |          |
|    actor_loss      | 27.3     |
|    critic_loss     | 0.551    |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 33.2     |
|    ep_rew_mean     | -28.9    |
| time/              |          |
|    episodes        | 271600   |
|    fps             | 518      |
|    time_elapsed    | 19214    |
|    total_timesteps | 9957680  |
| train/             |          |
|    actor_loss      | 26.8     |
|    critic_loss     | 0.46     |
|    ent_coef        | 0.0594   |
|    ent_coef_loss   | -0.267   |
|    learning_rate   | 0.0001   |
|    n_updates       | 622348   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.9     |
|    ep_rew_mean     | -29.7    |
| time/              |          |
|    episodes        | 271700   |
|    fps             | 518      |
|    time_elapsed    | 19220    |
|    total_timesteps | 9960944  |
| train/             |          |
|    actor_loss      | 26.8     |
|    critic_loss     | 0.396    |
|    ent_coef 

In [6]:
model.save("trained_models/g1-reptile/sac")